In [ ]:
from datasets import load_dataset, concatenate_datasets
from transformers import T5Tokenizer, T5ForConditionalGeneration, Trainer, TrainingArguments
import tqdm as notebook_tqdm

In [ ]:
import os
os.environ["HF_HOME"] = "F:/huggingface"
os.environ["HF_DATASETS_CACHE"] = "F:/huggingface/datasets"
os.environ["TRANSFORMERS_CACHE"] = "F:/huggingface/models"

In [13]:
ted_dataset = load_dataset("ted_talks_iwslt", "nl_en_2016", split="train", trust_remote_code=True)

subs_dataset = load_dataset("open_subtitles", "en-hi", split="train", trust_remote_code=True)

book_dataset = load_dataset("bookcorpus", split="train", trust_remote_code=True)

cnn_dataset = load_dataset("cnn_dailymail", "3.0.0", split="train", trust_remote_code=True)

Generating train split:   0%|          | 0/3852 [00:00<?, ? examples/s]

README.md:   0%|          | 0.00/7.45k [00:00<?, ?B/s]

open_subtitles.py:   0%|          | 0.00/6.22k [00:00<?, ?B/s]

Generating train split:   0%|          | 0/93016 [00:00<?, ? examples/s]

README.md:   0%|          | 0.00/18.5k [00:00<?, ?B/s]

...

validation-00000-of-00001.parquet:   0%|          | 0.00/34.7M [00:00<?, ?B/s]

test-00000-of-00001.parquet:   0%|          | 0.00/30.0M [00:00<?, ?B/s]

Generating train split:   0%|          | 0/287113 [00:00<?, ? examples/s]

Generating validation split:   0%|          | 0/13368 [00:00<?, ? examples/s]

Generating test split:   0%|          | 0/11490 [00:00<?, ? examples/s]

In [14]:
def format_ted(example):
    return {
        "input": "Generate a professional speech about: " + example["translation"]["en"][:60],
        "target": example["translation"]["en"]
    }

def format_subs(example):
    return {
        "input": "Generate a funny speech about: " + example["translation"]["en"][:60],
        "target": example["translation"]["en"]
    }

def format_book(example):
    return {
        "input": "Generate a structured speech from: " + example["text"][:60],
        "target": example["text"]
    }

def format_cnn(example):
    return {
        "input": "Generate a formal speech about: " + example["article"][:60],
        "target": example["article"]
    }


In [15]:
ted_formatted = ted_dataset.map(format_ted)
subs_formatted = subs_dataset.map(format_subs)
book_formatted = book_dataset.map(format_book)
cnn_formatted = cnn_dataset.map(format_cnn)

Map:   0%|          | 0/3852 [00:00<?, ? examples/s]

Map:   0%|          | 0/93016 [00:00<?, ? examples/s]

Map:   0%|          | 0/74004228 [00:00<?, ? examples/s]

Map:   0%|          | 0/287113 [00:00<?, ? examples/s]

In [16]:
# Only keep input and target
ted_formatted = ted_formatted.remove_columns([col for col in ted_formatted.column_names if col not in ["input", "target"]])
subs_formatted = subs_formatted.remove_columns([col for col in subs_formatted.column_names if col not in ["input", "target"]])
book_formatted = book_formatted.remove_columns([col for col in book_formatted.column_names if col not in ["input", "target"]])
cnn_formatted = cnn_formatted.remove_columns([col for col in cnn_formatted.column_names if col not in ["input", "target"]])



In [17]:
combined_dataset = concatenate_datasets([ted_formatted, subs_formatted, book_formatted, cnn_formatted])
print("Columns:", combined_dataset.column_names)
print("Features:", combined_dataset.features)
print("Number of examples:", len(combined_dataset))
print("First example:", combined_dataset[0])



Columns: ['input', 'target']
Features: {'input': Value(dtype='string', id=None), 'target': Value(dtype='string', id=None)}
Number of examples: 74388209
First example: {'input': "Generate a professional speech about: We don't have to live in a world where 99 percent of rapists", 'target': 'We don\'t have to live in a world where 99 percent of rapists get away with it, says TED Fellow Jessica Ladd. With Callisto, a new platform for college students to confidentially report sexual assault, Ladd is helping survivors get the support and justice they deserve while respecting their privacy concerns. "We can create a world where there\'s a real deterrent to violating the rights of another human being," she says.'}


In [18]:
tokenizer = T5Tokenizer.from_pretrained("t5-base")

spiece.model:   0%|          | 0.00/792k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.39M [00:00<?, ?B/s]

config.json:   0%|          | 0.00/1.21k [00:00<?, ?B/s]

In [19]:
def tokenize(batch, tokenizer):  # Add tokenizer as a parameter
    inputs = tokenizer(batch["input"], padding="max_length", truncation=True, max_length=128)
    targets = tokenizer(batch["target"], padding="max_length", truncation=True, max_length=128)
    inputs["labels"] = targets["input_ids"]  # Set the target input_ids as labels
    return inputs

In [20]:
tokenized_datasets = combined_dataset.map(tokenize, batched=True, batch_size=1000, num_proc=6, fn_kwargs={"tokenizer": tokenizer})

Map (num_proc=6):   0%|          | 0/74388209 [00:00<?, ? examples/s]

In [21]:
print(tokenized_datasets[100])

{'input': "Generate a professional speech about: When someone asks you where you're from … do you sometimes n", 'target': 'When someone asks you where you\'re from … do you sometimes not know how to answer? Writer Taiye Selasi speaks on behalf of "multi-local" people, who feel at home in the town where they grew up, the city they live now and maybe another place or two. "How can I come from a country?" she asks. "How can a human being come from a concept?"', 'input_ids': [6939, 2206, 3, 9, 771, 5023, 81, 10, 366, 841, 987, 7, 25, 213, 25, 31, 60, 45, 3, 233, 103, 25, 1664, 3, 29, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0], 'attention_mask': [1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 0, 

In [22]:
model_name = "t5-base"  
tokenizer = T5Tokenizer.from_pretrained(model_name)
model = T5ForConditionalGeneration.from_pretrained(model_name)

Xet Storage is enabled for this repo, but the 'hf_xet' package is not installed. Falling back to regular HTTP download. For better performance, install the package with: `pip install huggingface_hub[hf_xet]` or `pip install hf_xet`


model.safetensors:   0%|          | 0.00/892M [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/147 [00:00<?, ?B/s]

In [23]:
import transformers
print(transformers.__version__)


4.51.2


In [ ]:
training_args = TrainingArguments(
    output_dir="./t5-base-speech-gen",
    per_device_train_batch_size=16,
    gradient_accumulation_steps=2,
    num_train_epochs=3,
    learning_rate=1e-4,
    save_total_limit=2,
    save_steps=500,
    logging_steps=250,
    fp16=True,
    max_steps=3000,
    do_eval=False,
    resume_from_checkpoint="./t5-base-speech-gen/checkpoint-2000"  
)



trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=tokenized_datasets,
    tokenizer=tokenizer
)


C:\Users\aadhi\AppData\Local\Temp\ipykernel_37184\3419059096.py:18: FutureWarning: `tokenizer` is deprecated and will be removed in version 5.0.0 for `Trainer.__init__`. Use `processing_class` instead.
  trainer = Trainer(


In [43]:
import os

base_path = "./t5-base-speech-gen"
checkpoints = [ckpt for ckpt in os.listdir(base_path) if ckpt.startswith("checkpoint")]
print(sorted(checkpoints))


['checkpoint-1500', 'checkpoint-2000']


In [ ]:
trainer.train()

Step,Training Loss
250,0.128700
500,0.147500
750,0.149300
1000,0.140900


In [ ]:
model.save_pretrained("t5-base-speech-gen-model")
tokenizer.save_pretrained("t5-base-speech-gen-model")


In [ ]:
model = T5ForConditionalGeneration.from_pretrained("t5-base-speech-gen-model")
tokenizer = T5Tokenizer.from_pretrained("t5-base-speech-gen-model")



In [ ]:
prompt = "Generate a funny speech about artificial intelligence in college life"


In [ ]:
inputs = tokenizer(prompt, return_tensors="pt").input_ids.to(model.device)
output_ids = model.generate(inputs, max_length=200, num_beams=4, early_stopping=True)

In [ ]:
output = tokenizer.decode(output_ids[0], skip_special_tokens=True)
print("\nGenerated Speech:\n")
print(output)